In [19]:
from google.colab import drive
# Google Drive를 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
#폴더 만들기 네임 '/.kaggle'
!mkdir ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [21]:
!cp "/content/drive/MyDrive/kaggle API/kaggle.json" ~/.kaggle/kaggle.json

In [22]:
#set permission to the file
!chmod 600 ~/.kaggle/kaggle.json

In [23]:
# Install the kaggle python package using pip
!python -m pip install -qq kaggle

In [24]:
#download the dataset from kaggle
!kaggle datasets download -d stunningvisionai/coffee-leaf-diseases-yolo

100% 183M/184M [00:09<00:00, 24.3MB/s]
100% 184M/184M [00:09<00:00, 19.9MB/s]


In [25]:
#Unzip the dataset file
!unzip -qq "/content/coffee-leaf-diseases-yolo.zip"

In [26]:
import torch
from PIL import Image
!git clone https://github.com/ultralytics/yolov5

# 리포지토리의 디렉토리로 이동합니다.
%cd yolov5

# 의존성들을 설치합니다.
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16094, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 16094 (delta 0), reused 4 (delta 0), pack-reused 16089
Receiving objects: 100% (16094/16094), 14.81 MiB | 14.68 MiB/s, done.
Resolving deltas: 100% (11019/11019), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency con

In [27]:
!pip install pyyaml

In [28]:
import os
import shutil
from sklearn.model_selection import train_test_split
import yaml

# 데이터셋 경로 설정
dataset_path = '/content/coffee-leaf-diseases-yolo'
original_images_path = dataset_path  # 이미지 파일이 저장된 원본 경로
original_labels_path = dataset_path  # 라벨 파일이 저장된 원본 경로

# images, labels 서브 폴더 생성
images_path = os.path.join(dataset_path, 'images')
labels_path = os.path.join(dataset_path, 'labels')
train_images_path = os.path.join(images_path, 'train')
train_labels_path = os.path.join(labels_path, 'train')
val_images_path = os.path.join(images_path, 'val')
val_labels_path = os.path.join(labels_path, 'val')

os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)
os.makedirs(val_labels_path, exist_ok=True)

# 전체 이미지와 라벨 파일 목록을 가져옵니다.
image_files = [os.path.join(original_images_path, f) for f in os.listdir(original_images_path) if f.endswith('.jpg')]
label_files = [f.replace('.jpg', '.txt') for f in image_files]

# 학습 및 검증 데이터셋으로 분리
train_images, val_images, train_labels, val_labels = train_test_split(image_files, label_files, test_size=0.2, random_state=42)

# 분리된 데이터셋을 해당 폴더로 이동
for f in train_images:
    shutil.move(f, train_images_path)

for f in train_labels:
    shutil.move(f, train_labels_path)

for f in val_images:
    shutil.move(f, val_images_path)

for f in val_labels:
    shutil.move(f, val_labels_path)


In [29]:
print("Files in images folder:", os.listdir(images_path))
print("Files in labels folder:", os.listdir(labels_path))

Files in images folder: ['val', 'train']
Files in labels folder: ['val', 'train']


In [33]:
# 클래스 이름 설정 (예시)
class_names = ['0','1', '2','3', '4', '5'] # 실제 클래스 이름으로 교체

# data.yaml 파일 생성
data_yaml_content = {
    'train': train_images_path,
    'val': val_images_path,
    'nc': len(class_names),
    'names': class_names
}

# yaml 파일 쓰기
with open(os.path.join(dataset_path, 'data.yaml'), 'w') as yaml_file:
    yaml.dump(data_yaml_content, yaml_file)

In [34]:
# 학습 및 검증 세트의 이미지와 라벨 파일 수 확인
num_train_images = len(os.listdir(train_images_path))
num_val_images = len(os.listdir(val_images_path))
num_train_labels = len(os.listdir(train_labels_path))
num_val_labels = len(os.listdir(val_labels_path))

num_train_images, num_val_images, num_train_labels, num_val_labels

(1331, 333, 1331, 333)

In [35]:
!python train.py --img 640 --batch 16 --epochs 30 --data /content/coffee-leaf-diseases-yolo/data.yaml --weights yolov5s.pt --cache


2023-12-13 08:17:01.929392: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 08:17:01.929457: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 08:17:01.929497: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/coffee-leaf-diseases-yolo/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sy

In [36]:
!ls /content/yolov5/runs/train

exp  exp2


In [46]:
from google.colab import drive
import torch
from PIL import Image
import matplotlib.pyplot as plt

# Google Drive를 마운트합니다.
drive.mount('/content/drive', force_remount=True)


# 이미지 경로 설정
img_path = '/content/drive/My Drive/커피잎병.jpeg'  # Google Drive에서 추론하려는 이미지의 경로
#img_path = '/content/drive/My Drive/건강잎.jpeg'  # Google Drive에서 추론하려는 이미지의 경로
# YOLOv5 detect.py 스크립트 실행
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --img 640 --conf 0.25 --source "{img_path}"





Mounted at /content/drive
detect: weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], source=/content/drive/My Drive/커피잎병.jpeg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-249-gf400bba Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/drive/My Drive/커피잎병.jpeg: 448x640 5 0s, 44.2ms
Speed: 0.5ms pre-process, 44.2ms inference, 1.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp10
